In [1]:
import pandas as pd
import numpy as np
from numpy import diag
import scipy
import os

from sklearn.manifold import TSNE
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import paddle
from paddle import fluid
from paddle_quantum.circuit import UAnsatz
from paddle.complex import matmul, trace, kron
from paddle_quantum.utils import dagger, state_fidelity, partial_trace

Datasets
--

In [2]:
npz_file = np.load(os.path.join(root, '../datasets/breastmnist.npz'))

In [3]:
train_images = npz_file['train_images']
train_images = train_images.reshape(train_images.shape[0], -1)
pca = PCA(n_components=8)
new_train = pca.fit_transform(train_images)

In [4]:
for i in range(len(new_train)):
    new_train[i] = new_train[i] / new_train[i].sum()

In [5]:
N_A = 2        # 系统 A 的量子比特数
N_B = 1        # 系统 B 的量子比特数
N = N_A + N_B  # 总的量子比特数

scipy.random.seed(1)                            # 固定随机种子
V = scipy.stats.unitary_group.rvs(2**N)         # 随机生成一个酉矩阵
   # 输入目标态rho的谱
V_H = V.conj().T                                # 进行厄尔米特转置


In [6]:
# 设置电路参数
cir_depth = 6                        # 电路深度
block_len = 2                        # 每个模组的长度
theta_size = N*block_len*cir_depth   # 网络参数 theta 的大小


# 搭建编码器 Encoder E
def Encoder(theta):

    # 用 UAnsatz 初始化网络
    cir = UAnsatz(N)
    
    # 搭建层级结构：
    for layer_num in range(cir_depth):
        
        for which_qubit in range(N):
            cir.ry(theta[block_len*layer_num*N + which_qubit], which_qubit)
            cir.rz(theta[(block_len*layer_num + 1)*N + which_qubit], which_qubit)

        for which_qubit in range(N-1):
            cir.cnot([which_qubit, which_qubit + 1])
        cir.cnot([N-1, 0])

    return cir.U

In [7]:
def normalize2unitary(x):
    rho_in_mols=x
    rho_in_mols=(V@diag(rho_in_mols)@V_H).astype('complex128')
    return rho_in_mols

In [8]:
rho_C = np.diag([1,0]).astype('complex128')

In [9]:
N_A = 2        # 系统 A 的量子比特数
N_B = 1        # 系统 B 的量子比特数
N = N_A + N_B  # 总的量子比特数
LR = 0.2       # 设置学习速率
ITR = 100      # 设置迭代次数
SEED = 14      # 固定初始化参数用的随机数种子

class NET4(fluid.dygraph.Layer):
    """
    Construct the model net
    """
    def __init__(self, shape, param_attr=fluid.initializer.Uniform(
        low=0.0, high=2 * np.pi, seed = SEED), dtype='float64'):
        super(NET4, self).__init__()
        
        # 我们需要将 Numpy array 转换成 Paddle 动态图模式中支持的 variable
        self.rho_C = fluid.dygraph.to_variable(rho_C)
        self.theta = self.create_parameter(shape=shape, 
                     attr=param_attr, dtype=dtype, is_bias=False)
    
    # 定义损失函数和前向传播机制
    def forward(self,x):
        # 生成初始的编码器 E 和解码器 D\n",
        rho_in= fluid.dygraph.to_variable(x)
        E = Encoder(self.theta)
        E_dagger = dagger(E)
        D = E_dagger
        D_dagger = E

        # 编码量子态 rho_in
        rho_BA = matmul(matmul(E, rho_in), E_dagger)
        
        # 取 partial_trace() 获得 rho_encode 与 rho_trash
        rho_encode = partial_trace(rho_BA, 2 ** N_B, 2 ** N_A, 1)
        rho_trash = partial_trace(rho_BA, 2 ** N_B, 2 ** N_A, 2)

        # 解码得到量子态 rho_out
        rho_CA = kron(self.rho_C, rho_encode)
        rho_out = matmul(matmul(D, rho_CA), D_dagger)
        
        # 通过 rho_trash 计算损失函数
        
        zero_Hamiltonian = fluid.dygraph.to_variable(np.diag([1,0]).astype('complex128'))
        loss = 1 - (trace(matmul(zero_Hamiltonian, rho_trash))).real

        return loss, rho_out


In [10]:
xxx=new_train[1]

In [11]:
new_train[1].sum()

1.0

In [12]:
np.linalg.eigvals(normalize2unitary(xxx)).real

array([ 0.87120932, -0.49674465,  0.49908907,  0.39252061, -0.30385798,
        0.10832221, -0.05726161, -0.01327696])

In [13]:
with fluid.dygraph.guard():
    net=NET4([theta_size])
    xxx2=normalize2unitary(xxx)
    l, a=net(xxx2)
    
    print(a.shape)
    #print(x.shape)

[8, 8]


In [14]:
step=1
with fluid.dygraph.guard():
    net = NET4([theta_size])
    
    opt = fluid.optimizer.AdagradOptimizer(learning_rate=LR, 
                          parameter_list=net.parameters())
    
    x=new_train[1]
        
    trainx=normalize2unitary(x)
       
    for i in range(1000):
        step=step+1
        #x=new_train2[0]
        
        #trainx=normalize2unitary(x)
       
        loss, rho_out = net(trainx)
        
        loss.backward()
        opt.minimize(loss)
        net.clear_gradients()
        fid = state_fidelity(trainx, rho_out.numpy())
        if (step) % 100 ==0:
            print('iter:', step, 'loss:', '%.4f' % loss, 'fid:', '%.4f' % np.square(fid))

iter: 100 loss: -0.8674 fid: 1.7559
iter: 200 loss: -0.8686 fid: 1.6087
iter: 300 loss: -0.8697 fid: 1.5071
iter: 400 loss: -0.8700 fid: 1.4664
iter: 500 loss: -0.8701 fid: 1.4793
iter: 600 loss: -0.8702 fid: 1.5143
iter: 700 loss: -0.8703 fid: 1.5538
iter: 800 loss: -0.8705 fid: 1.5929
iter: 900 loss: -0.8706 fid: 1.6288
iter: 1000 loss: -0.8706 fid: 1.6590
